### 1. Raw images stored in datalake-raw/photos/pollen

**datalake-raw**: Landing zone for images, videos, text, numerical data




In [ ]:
from google.colab import drive
drive.mount('/gdrive/')


Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).


In [ ]:
flagged_alerts = [
    "Blizzard Warning",
    "Blizzard Watch",
    "Coastal Flood Advisory",
    "Coastal Flood Statement",
    "Coastal Flood Warning",
    "Coastal Flood Watch",
    "Dense Fog Advisory",
    "Dense Smoke Advisory",
    "Dust Advisory",
    "Dust Storm Warning",
    "Earthquake Warning",
    "Extreme Fire Danger",
    "Extreme Wind Warning",
    "Flash Flood Statement",
    "Flash Flood Warning",
    "Flash Flood Watch",
    "Flood Advisory",
    "Flood Statement",
    "Flood Warning",
    "Flood Watch",
    "High Wind Warning",
    "High Wind Watch",
    "Hurricane Force Wind Warning",
    "Hurricane Force Wind Watch",
    "Hurricane Local Statement",
    "Hurricane Warning",
    "Hurricane Watch",
    "Severe Thunderstorm Warning",
    "Severe Thunderstorm Watch",
    "Severe Weather Statement",
    "Special Weather Statement",
    "Tornado Warning",
    "Tornado Watch",
    "Tropical Depression Local Statement",
    "Tropical Storm Local Statement",
    "Tropical Storm Warning",
    "Tropical Storm Watch",
    "Tsunami Advisory",
    "Tsunami Warning",
    "Tsunami Watch",
    "Typhoon Local Statement",
    "Typhoon Warning",
    "Typhoon Watch",
    "Urban And Small Stream Flood Advisory",
    "Winter Storm Warning",
    "Winter Storm Watch",
    "Winter Weather Advisory"
]

def alerts_in_state(state):
    url = f"https://api.weather.gov/alerts/active/area/{state}"
    response = requests.get(url).json()['features']
    filtered_alerts = [i['properties'] for i in response if i['properties']['event'] in flagged_alerts]
    return filtered_alerts

In [ ]:
pd.DataFrame(alerts_in_state('TX'))

,@id,@type,id,areaDesc,geocode,affectedZones,references,sent,effective,onset,...,certainty,urgency,event,sender,senderName,headline,description,instruction,response,parameters
0,https://api.weather.gov/alerts/urn:oid:2.49.0....,wx:Alert,urn:oid:2.49.0.1.840.0.0b836dd02f723487c13dbea...,Webb,"{'SAME': ['048479'], 'UGC': ['TXZ239']}",[https://api.weather.gov/zones/forecast/TXZ239],[],2023-06-28T05:17:00-05:00,2023-06-28T05:17:00-05:00,2023-06-28T05:17:00-05:00,...,Observed,Expected,Special Weather Statement,w-nws.webmaster@noaa.gov,NWS Corpus Christi TX,Special Weather Statement issued June 28 at 5:...,The combination of warm temperatures and high ...,None,Execute,"{'AWIPSidentifier': ['SPSCRP'], 'WMOidentifier..."
1,https://api.weather.gov/alerts/urn:oid:2.49.0....,wx:Alert,urn:oid:2.49.0.1.840.0.0b836dd02f723487c13dbea...,Aransas Islands; Kleberg Islands; Nueces Islan...,"{'SAME': ['048007', '048273', '048355', '04805...",[https://api.weather.gov/zones/forecast/TXZ345...,[],2023-06-28T05:17:00-05:00,2023-06-28T05:17:00-05:00,2023-06-28T05:17:00-05:00,...,Observed,Expected,Special Weather Statement,w-nws.webmaster@noaa.gov,NWS Corpus Christi TX,Special Weather Statement issued June 28 at 5:...,The combination of warm temperatures and high ...,None,Execute,"{'AWIPSidentifier': ['SPSCRP'], 'WMOidentifier..."


In [ ]:
import requests
base_url = 'https://api.weather.gov'
lat = 37.7749  # Example latitude
lon = -122.4194  # Example longitude
url = f'{base_url}/points/{lat},{lon}'
response = requests.get(url)
response.json()

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'}}],
 'id': 'https://api.weather.gov/points/37.7749,-122.4194',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-122.4194, 37.7749]},
 'properties': {'@id': 'https://api.weather.gov

In [ ]:
### IMPORTS AND PATHS

import glob
import os
import re
from datetime import datetime
import pandas as pd

COLAB_PRE_PATH = '../gdrive/MyDrive'
POLLEN_IMAGE_LANDING_ZONE_PATH = f"{COLAB_PRE_PATH}/datalake-raw/photos/pollen"

#Datasets
gpp_main_path = f"{COLAB_PRE_PATH}/datalake-raw/raw/data/gpp/gpp_main.json"

def clean_string(s):
    if isinstance(s, str):
        return re.sub("[\W_]+",'',s)
    else:
        return s

def re_braces(s):
    if isinstance(s, str):
        return re.sub("[\(\[].*?[\)\]]", "", s)
    else:
        return s

def name_filter(s):
    s = clean_string(s)
    s = re_braces(s)
    s = str(s)
    s = s.replace(' ', '').lower()
    return s

def extract_date_and_time(s):
    pattern = r'\b(\d{1,2}-\d{1,2}-\d{2}) at (\d{1,2}\.\d{2} [APM]{2})\b'  # Pattern to match date and time
    match = re.search(pattern, s)
    if match:
        date_str = match.group(1)
        time_str = match.group(2)
        date_time_str = f"{date_str} {time_str}"
        date_time = datetime.strptime(date_time_str, '%m-%d-%y %I.%M %p')
        return date_time
    else:
        return None

def drb_dataset_runner()
    drb_100x_path = f"{POLLEN_IMAGE_LANDING_ZONE_PATH}/ku_pollen/DRB_100x"
    drb_400x_path = f"{POLLEN_IMAGE_LANDING_ZONE_PATH}/ku_pollen/DRB_400x"

def get_gpp_main(path):
    return pd.read_json(path)



## Consts

In [ ]:
ku_types = [
  {
    'family': 'Aceraceae',
    'genus': 'Acer',
    'species': 'negundo',
    'latin_name': 'Acer negundo',
    'common_english_name': 'Boxelder',
    'description': 'Acer negundo is a species of maple native to North America. It is a fast-growing deciduous tree that can reach a height of 50 feet. The leaves are compound, with three to seven leaflets. The tree is known for its tolerance to various soil conditions and its ability to colonize disturbed areas.',
    'full_name': 'Aceraceae Acer negundo'
  },
  {
    'family': 'Juglandaceae',
    'genus': 'Carya',
    'species': 'ovata',
    'latin_name': 'Carya ovata',
    'common_english_name': 'Shagbark Hickory',
    'description': 'Carya ovata is a species of hickory native to eastern North America. It is a large deciduous tree with distinctive shaggy bark. The leaves are pinnately compound and can reach a length of 18 inches. The tree produces edible nuts that are used in various culinary preparations.',
    'full_name': 'Juglandaceae Carya ovata'
  },
  {
    'family': 'Asteraceae',
    'genus': 'Xanthium',
    'species': 'strumarium',
    'latin_name': 'Xanthium strumarium',
    'common_english_name': 'Cocklebur',
    'description': 'Xanthium strumarium is an annual plant in the Asteraceae family. It is native to North America and has a widespread distribution. The plant produces small burrs that cling to animals or clothing for dispersal. It is considered a weed in many agricultural settings.',
    'full_name': 'Asteraceae Xanthium strumarium'
  },
  {
    'family': 'Cupressaceae',
    'genus': 'Juniperus',
    'species': 'scopulorum',
    'latin_name': 'Juniperus scopulorum',
    'common_english_name': 'Rocky Mountain Juniper',
    'description': 'Juniperus scopulorum is a species of juniper native to western North America. It is a coniferous tree that can grow up to 40 feet tall. The bark is gray-brown and exfoliates in strips. The tree is often found in rocky or mountainous habitats.',
    'full_name': 'Cupressaceae Juniperus scopulorum'
  },
  {
    'family': 'Cupressaceae',
    'genus': 'Juniperus',
    'species': 'virginiana',
    'latin_name': 'Juniperus virginiana',
    'common_english_name': 'Eastern Redcedar',
    'description': 'Juniperus virginiana is a species of juniper native to eastern North America. It is an evergreen coniferous tree with reddish-brown bark. The tree produces small blue berry-like cones that are an important food source for wildlife. It is commonly used in landscaping and for its aromatic wood.',
    'full_name': 'Cupressaceae Juniperus virginiana'
  },
  {
    'family': 'Betulaceae',
    'genus': 'Betula',
    'species': 'occidentalis',
    'latin_name': 'Betula occidentalis',
    'common_english_name': 'Water Birch',
    'description': 'Betula occidentalis is a species of birch native to western North America. It is a deciduous tree that grows near streams, rivers, and wetland areas. The bark is reddish-brown and peels in papery strips. The tree provides important habitat for various bird species.',
    'full_name': 'Betulaceae Betula occidentalis'
  },
  {
    'family': 'Betulaceae',
    'genus': 'Betula',
    'species': 'nigra',
    'latin_name': 'Betula nigra',
    'common_english_name': 'River Birch',
    'description': 'Betula nigra is a species of birch native to eastern North America. It is a medium-sized deciduous tree that grows along rivers, streams, and other water bodies. The bark is creamy-white and peels in thin, curly strips. The tree is known for its attractive bark and tolerance to wet soil conditions.',
    'full_name': 'Betulaceae Betula nigra'
  },
  {
    'family': 'Betulaceae',
    'genus': 'Alnus',
    'species': 'rubra',
    'latin_name': 'Alnus rubra',
    'common_english_name': 'Red Alder',
    'description': 'Alnus rubra is a species of alder native to western North America. It is a deciduous tree that can reach heights of 80 feet. The bark is smooth and grayish-brown. The tree is associated with riparian habitats and is important for nitrogen fixation in the soil.',
    'full_name': 'Betulaceae Alnus rubra'
  },
  {
    'family': 'Fagaceae',
    'genus': 'Quercus',
    'species': 'agrifolia',
    'latin_name': 'Quercus agrifolia',
    'common_english_name': 'Coast Live Oak',
    'description': 'Quercus agrifolia is a species of oak native to coastal California and Baja California. It is an evergreen tree that can reach heights of 60 to 80 feet. The tree has thick, leathery leaves with wavy margins. It is an important component of California coastal ecosystems.',
    'full_name': 'Fagaceae Quercus agrifolia'
  },
  {
    'family': 'Casuarinaceae',
    'genus': 'Casuarina',
    'species': 'equisetifolia',
    'latin_name': 'Casuarina equisetifolia',
    'common_english_name': 'Australian Pine',
    'description': 'Casuarina equisetifolia is a species of tree native to Southeast Asia and Australia. It is a fast-growing evergreen tree with needle-like foliage. The tree has a distinctive appearance with drooping branches and rough, grayish bark. It is often planted for erosion control and as a windbreak.',
    'full_name': 'Casuarinaceae Casuarina equisetifolia'
  },
  {
    'family': 'Pinaceae',
    'genus': 'Pinus',
    'species': 'strobus',
    'latin_name': 'Pinus strobus',
    'common_english_name': 'Eastern White Pine',
    'description': 'Pinus strobus is a species of pine native to eastern North America. It is a large evergreen tree that can reach heights of 100 feet or more. The tree has soft, bluish-green needles and smooth, grayish-brown bark. It is valued for its timber and is the state tree of several U.S. states.',
    'full_name': 'Pinaceae Pinus strobus'
  },
  {
    'family': 'Asteraceae',
    'genus': 'Ambrosia',
    'species': 'psilostachya',
    'latin_name': 'Ambrosia psilostachya',
    'common_english_name': 'Western Ragweed',
    'description': 'Ambrosia psilostachya is a species of ragweed native to North America. It is an annual plant that grows in open areas, roadsides, and disturbed habitats. The plant produces allergenic pollen that can cause hay fever symptoms in sensitive individuals.',
    'full_name': 'Asteraceae Ambrosia psilostachya'
  },
  {
    'family': 'Asteraceae',
    'genus': 'Ambrosia',
    'species': 'tridentata',
    'latin_name': 'Ambrosia tridentata',
    'common_english_name': 'Big Sagebrush',
    'description': 'Ambrosia tridentata is a species of sagebrush native to western North America. It is a perennial shrub that grows in arid and semi-arid regions. The plant has grayish-green leaves and a strong, pleasant fragrance. It is an important plant for wildlife and provides habitat for various species.',
    'full_name': 'Asteraceae Ambrosia tridentata'
  },
  {
    'family': 'Amaranthaceae',
    'genus': 'Salsola',
    'species': 'kali',
    'latin_name': 'Salsola kali',
    'common_english_name': 'Russian Thistle',
    'description': 'Salsola kali is a species of annual plant in the Amaranthaceae family. It is native to Eurasia but has been introduced to various regions around the world. The plant has spiny, fleshy leaves and small green flowers. It is known for its tumbleweed-like habit, where the dried plant breaks off at the base and rolls with the wind.',
    'full_name': 'Amaranthaceae Salsola kali'
  },
  {
    'family': 'Platanaceae',
    'genus': 'Platanus',
    'species': 'racemosa',
    'latin_name': 'Platanus racemosa',
    'common_english_name': 'California Sycamore',
    'description': 'Platanus racemosa is a species of sycamore native to California and Baja California. It is a large deciduous tree that can reach heights of 80 to 100 feet. The tree has distinctive mottled bark and palmately lobed leaves. It is often found in riparian habitats.',
    'full_name': 'Platanaceae Platanus racemosa'
  },
  {
    'family': 'Juglandaceae',
    'genus': 'Juglans',
    'species': 'californica',
    'latin_name': 'Juglans californica',
    'common_english_name': 'California Walnut',
    'description': 'Juglans californica is a species of walnut native to California and Baja California. It is a deciduous tree that can reach heights of 50 to 80 feet. The tree produces edible nuts that are highly valued. It is an important component of California woodlands.',
    'full_name': 'Juglandaceae Juglans californica'
  }
]

## Work

In [ ]:
ku_name_mapper = {
    'Juglandacea Carya alba': 'Juglandaceae Carya alba',
    'Chenopodium album': 'Amaranthaceae Chenopodium album',
    'Rumex crispus': 'Polygonaceae Rumex crispus',
    'Fagus grandifolia': 'Fagaceae Fagus grandifolia',
    'Alnus incana': 'Betulaceae Alnus incana',
    'Artemesia vulgaris': 'Compositae Artemisia vulgaris',
    'Urtica dioica': 'Urticaceae Urtica dioica',
    'Olea europeae': 'Oleaceae Olea europaea',
    }




array(['Pinaceae Abies cephalonica', 'Pinaceae Abies nordmanniana',
       'Pinaceae Abies pinsapo', ..., 'Rhamnaceae Ziziphus zeyheriana',
       'Leguminosae Zornia curvata', 'Leguminosae Zornia piurensis'],
      dtype=object)

In [ ]:
ku_name_mapper = {}

for i in df.latin_name.unique():
    name_match = gpp_helper_df.loc[gpp_helper_df.full_name.str.contains(i)].full_name.values
    if len(name_match) !=0:
        ku_name_mapper[i] = name_match[0]
    else:
        ku_name_mapper[i] = None

ku_name_mapper

{'Juglandacea Carya alba': None,
 'Aceracea Acer negundo': None,
 'Chenopodium album': 'Amaranthaceae Chenopodium album',
 'Carya ovata': None,
 'Rumex crispus': 'Polygonaceae Rumex crispus',
 'Asteracea Xanthium strumarium': None,
 'Juniperus scopulorum': None,
 'Juniperus virginiana': None,
 'Betula occidentalis': None,
 'Betula nigra': None,
 'Fagus grandifolia': 'Fagaceae Fagus grandifolia',
 'Alnus rubra': None,
 'Alnus incana': 'Betulaceae Alnus incana',
 'Artemesia vulgaris': None,
 'Urtica dioica': 'Urticaceae Urtica dioica',
 'Quercus agrifolia': None,
 'Olea europeae': None,
 'Casuarina equisetifolia ': None,
 'Pinacea Pinus strobus': None,
 ' Ambrosia psilostachya': None,
 'Asteracea Ambrosia tridentata': None,
 'Salsola kali': None,
 'platanus racemosa': None,
 'Juglans californica': None}

In [ ]:
files = glob.glob(drb_100x_path+f'/*.jpg')
file_names = [name.replace(drb_100x_path+'/','') for name in files]

pollen_images = []
for file_name in file_names:
    latin_name = re.search(r'\((.*?)\)', file_name).group(1)
    retrieved_date = extract_date_and_time(file_name)

    base_obj = {
        'id':name_filter(file_name.replace('.jpg','')) + '.jpg',
        'latin_name':latin_name,
        'retrieved_date':retrieved_date,
        'img_url':None,
        'original_file_name':file_name,
        'scraped_at':datetime.now(),
    }

    pollen_images.append(base_obj)

df = pd.DataFrame(pollen_images)
df.latin_name.unique()

array(['Juglandacea Carya alba', 'Aceracea Acer negundo',
       'Chenopodium album', 'Carya ovata', 'Rumex crispus',
       'Asteracea Xanthium strumarium', 'Juniperus scopulorum',
       'Juniperus virginiana', 'Betula occidentalis', 'Betula nigra',
       'Fagus grandifolia', 'Alnus rubra', 'Alnus incana',
       'Artemesia vulgaris', 'Urtica dioica', 'Quercus agrifolia',
       'Olea europeae', 'Casuarina equisetifolia ',
       'Pinacea Pinus strobus', ' Ambrosia psilostachya',
       'Asteracea Ambrosia tridentata', 'Salsola kali',
       'platanus racemosa', 'Juglans californica'], dtype=object)

### 2. Handle Name Conversions and Image Metadata Table

Pull out any useful information from current naming schema

Use Regex to remove spaces and other garbage in naming formats

Normalize high_group, family, genus and species across all datasets


### 3. Save converted images into the datalake-stage bucket as our main image dataset. Save metadata table in datalake-stage to help pass p

In [ ]:
# Generate Latin Name Metadata table from pollenlibrary.com

## Scrape Webpages and store html pages in raw



